In [1]:
from brainlit.utils.write import czi_to_zarr, zarr_to_omezarr

# Convert czi to zarr

In [ ]:
czi_path = "/cis/project/sriram/for Tomi/exp227-mouse3-stitched.czi"
out_dir = "/cis/project/sriram/ng_data/exp227/"

zarr_paths = czi_to_zarr(czi_path=czi_path, out_dir=out_dir)

# Convert zarr to ome-zarr

In [ ]:
out_path = "/cis/project/sriram/ng_data/exp227/fg_ome.zarr"

zarr_to_omezarr(zarr_path=zarr_paths[0], out_path=out_path)

# View ome-zarr on neuroglancer

1. Use https://github.com/google/neuroglancer/blob/master/cors_webserver.py to serve the data e.g. `python cors_webserver.py -d "/cis/project/sriram/ng_data/" -p 9010`
2. Launch neuroglancer app e.g. https://neuroglancer-demo.appspot.com
3. Add layer to neuroglancer app e.g. zarr://http://127.0.0.1:9010/exp227/fg_ome.zarr